In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import mysql.connector
import numpy as np


1) Вписать торговые идеи в БД
Проверять на наличе, если она есть, то тогда ничего не делать


In [3]:
CONFIG = {
    'user': 'tester',
    'password': 'Fx123456#',
    'host': '192.168.1.6',
    'port': '3306',
    'raise_on_warnings': True}
UNITY_KEY = 'c3f28d5b-9694-4307-b30f-6f800ee13506'

from process_data import load_data
import time
import json
with open('right_instumets.json', 'r', encoding='utf-8') as f:
    RIGHT_INSTRUMENTS = json.load(f)

In [4]:
connection = mysql.connector.connect(**CONFIG)
sql = '''
SELECT expiration_time, current_state, report,pl
FROM tradeideas.unity_data
WHERE 
expiration_time BETWEEN DATE_SUB(CURDATE(), INTERVAL 1 DAY) AND CURDATE();'''
df = pd.read_sql(sql,connection)
connection.close()
df

C:\Temp\ipykernel_28112\838399473.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,connection)


,expiration_time,current_state,report,pl
0,2024-06-04 05:00:00,NOT_ENTER,OK,NaN
1,2024-06-04 05:00:00,SL,OK,-0.0750
2,2024-06-04 05:00:00,NOT_ENTER,OK,NaN
3,2024-06-04 05:00:00,NOT_ENTER,OK,NaN
4,2024-06-04 05:00:00,None,Неизвестный запрос для Exante API,NaN
5,2024-06-04 05:00:00,None,Неизвестный запрос для Exante API,NaN
6,2024-06-04 05:00:00,SL,OK,-0.3500
7,2024-06-04 05:00:00,TP,OK,0.0044
8,2024-06-04 05:00:00,None,Неизвестный запрос для Exante API,NaN
9,2024-06-04 05:00:00,NOT_ENTER,OK,NaN


In [5]:
if not df.empty:
    print('+')

+


In [3]:
def Strategy_Tester(data:list, side:str, open_order_type:str, stop_loss:float, price:float, take_profit:float)->tuple:
    df = pd.DataFrame(data)
    # Преобразование timestamp в формат datetime с учетом часового пояса UTC
    df['timestamp2'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_localize('UTC')
    # Преобразуем дату в формат "d-m-Y H:M:S" и создадим новый столбец 'formatted_date'
    df['formatted_date'] = df['timestamp2'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df = df.drop(columns=["timestamp2"])
    df[["close", "high", "low", "open"]] = df[["close", "high", "low", "open"]].apply(pd.to_numeric, errors='coerce')
    df = df[::-1]
    df = df.reset_index(drop=True)
    '''Алгоритм проверки'''
    new_df = df.copy()
    PRICE_INDICATOR = False
    STOP_LOSS_INDICATOR = False
    TAKE_PROFIT_INDICATOR = False
    # 1. Проверяем вход по цене
    if side == 'Buy':
        Flag=True
        if open_order_type == 'Limit':
            price_open = new_df[new_df['low'] <= price]
        else:
            price_open = new_df[new_df['high'] >= price]
    else:
        Flag = False
        if open_order_type == 'Limit':
            price_open = new_df[new_df['high'] >= price]
        else:
            price_open = new_df[new_df['low'] <= price]

    if not price_open.empty:
        index_price = price_open.index[0]
        new_df = new_df.iloc[index_price:].reset_index(drop=True)
        PRICE_INDICATOR = True
        approximate_time_enter, accurate_time_enter = price_open['formatted_date'].iloc[0], price_open['timestamp'].iloc[0]
    # 2. Проверяем стоп-лосс
    if Flag:
        stop_loss_res = new_df[new_df['low'] <= stop_loss]
    else:
        stop_loss_res = new_df[new_df['high'] >= stop_loss]

    if not stop_loss_res.empty:
        index_stop_loss = stop_loss_res.index[0]
        new_df = new_df.iloc[:index_stop_loss + 1].reset_index(drop=True)
        STOP_LOSS_INDICATOR = True
        approximate_time_ST, accurate_time_ST = stop_loss_res['formatted_date'].iloc[0], stop_loss_res['timestamp'].iloc[0]

    # 3. Проверяем на take profit
    if Flag:
        take_profit_res = new_df[new_df['high'] >= take_profit]
    else:
        take_profit_res = new_df[new_df['low'] <= take_profit]
        
    if not take_profit_res.empty:
        take_profit_index = take_profit_res.index[0]
        TAKE_PROFIT_INDICATOR = True
        approximate_time_TP, accurate_time_TP = take_profit_res['formatted_date'].iloc[0], take_profit_res['timestamp'].iloc[0]

    def timedelta(str1, str2):
        date_format = '%Y-%m-%d %H:%M:%S'
        date1 = datetime.strptime(str1, date_format)
        date2 = datetime.strptime(str2, date_format)
        # Вычисление разницы между датами
        time_difference = date2 - date1 
        # Извлечение часов, минут и секунд из разницы
        hours, remainder = divmod(time_difference.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        # Форматирование результатов
        result_string = '{:02}:{:02}:{:02}'.format(hours, minutes, seconds)
        return result_string
        
    # 4. Вывод
    if PRICE_INDICATOR:
        if TAKE_PROFIT_INDICATOR:
            return("TP",take_profit, abs(take_profit-price), approximate_time_enter, approximate_time_TP, timedelta(approximate_time_enter,approximate_time_TP))
        elif STOP_LOSS_INDICATOR and not TAKE_PROFIT_INDICATOR:
            return ("SL",stop_loss, -1*abs(stop_loss-price), approximate_time_enter, approximate_time_ST, timedelta(approximate_time_enter,approximate_time_ST))
        else:
            last_price = new_df['close'].iloc[-1]
            approximate_time_EXP = new_df['formatted_date'].iloc[-1]
            return ("EXP", 
                    last_price, 
                    last_price - price if side == 'Buy'else price - last_price, 
                    approximate_time_enter, 
                    approximate_time_EXP, 
                    timedelta(approximate_time_enter,approximate_time_EXP))
    else:
        return('NOT_ENTER')

In [6]:
class Tester_Bot:
    def __init__(self) -> None:
        self.link_UNITY_select_all_active_ideas = 'https://rest.unity.finance/api/v1/tradeIdeas?limit=1000&statuses=ACTIVE'

        self.table_name = 'tradeideas.unity_data'

    def Unity_api(link:str) -> list:
        headers = {'auth-token' : UNITY_KEY}
        response = requests.get(link, headers=headers)
        return response.json()
    
    def main_process_data(self) -> pd.DataFrame:
        'Данный метод комплексно обрабатывает данные. Потом данные вставляються в БД, предварительно проверяться на наличе уже сделанной записи.'
        d = Tester_Bot.Unity_api(self.link_UNITY_select_all_active_ideas)
        df = pd.DataFrame(d['items'])
        df['expirationTime'] = (pd.to_datetime(df['expirationTime'], format="%Y-%m-%dT%H:%M:%SZ")).dt.strftime("%Y-%m-%d %H:%M:%S")
        df['createTime'] = (pd.to_datetime(df['createTime'], format="%Y-%m-%dT%H:%M:%SZ")).dt.strftime("%Y-%m-%d %H:%M:%S")
        df['instrumentIds'] = df['instrumentIds'].apply(lambda x: x[0])
        #===================================
        instrumentIds = df['instrumentIds'].tolist()
        instrumentIds = ",".join(map(str, instrumentIds))

        link = f'https://rest.unity.finance/api/v1/instrumentDetails?id={instrumentIds}'
        data = Tester_Bot.Unity_api(link)
        ticker_df = pd.DataFrame(data['items'])
        ticker_df = ticker_df[['id','code']]
        df = pd.merge(df, ticker_df, left_on='instrumentIds', right_on='id', how='left')
        df["instrumentIds"] = df['code']
        #============================

        df = df.drop(['brokerId','status', 'publishTime','updateTime','id_y','code'], axis=1)
        df.columns = [['unity_id','instrument','open_order_type','side', 'open_order_price','stop_loss', 'take_profit','confidence','expiration_time','description_text','create_time']]
        #============================
        table_name = 'tradeideas.unity_data'
        connection = mysql.connector.connect(**CONFIG)
        cursor = connection.cursor()

        cursor.execute('SELECT unity_id FROM tradeideas.unity_data')
        id_unity_spisok = np.array(cursor.fetchall()).flatten()

        columns_name = list(map(lambda x: x[0],df.columns.tolist()))
        for index, row in df.iterrows():
            unity_id = row['unity_id']
            if unity_id not in id_unity_spisok:
                values = [str(value) for value in row.tolist()]

                sql_query = f"INSERT INTO {table_name} ({', '.join(columns_name)}) VALUES ({', '.join(['%s']*len(columns_name))})"
                
                # Исполнение SQL-запроса
                cursor.execute(sql_query, tuple(values))
        connection.commit()
        cursor.close()
        connection.close()
    
    def take_necessary_data(self) -> list:
        'Шаг 2. Забираем нужные нам данные из БД и провекряем их'
        connection = mysql.connector.connect(**CONFIG)
        sql_query = '''
        SELECT id, instrument, side, open_order_type, open_order_price, stop_loss, take_profit, create_time, expiration_time
        FROM tradeideas.unity_data
        WHERE 
        expiration_time <= NOW() and 
        report is NULL;'''
        check_df = pd.read_sql(sql_query, connection)
        connection.close()
        # ИЗМЕНЯЕМ ВРЕМЯ НА НОРМАЛЬНОЕ 
        check_df['expiration_time'] = (pd.to_datetime(check_df['expiration_time'], format="%Y-%m-%d %H:%M:%S")).dt.strftime("%d.%m.%Y %H:%M:%S")
        check_df['create_time'] = (pd.to_datetime(check_df['create_time'], format="%Y-%m-%d %H:%M:%S")).dt.strftime("%d.%m.%Y %H:%M:%S")
        check_df.set_index('id', inplace=True)
        # Проверка данных
        strategy_result = []
        for idx, row in check_df.iterrows():
            instrument, side, open_order_type, open_order_price, stop_loss, take_profit, create_time, expiration_time  = row
            if instrument in RIGHT_INSTRUMENTS:
                instrument = RIGHT_INSTRUMENTS[instrument]
            try:
                data = load_data(instrument, create_time, expiration_time)
                result = Strategy_Tester(data, side, open_order_type, stop_loss, open_order_price, take_profit)
                strategy_result.append((idx, result, 'OK'))
                time.sleep(60)   
            except:
                time.sleep(60)
                strategy_result.append((idx, 'Неизвестный запрос для Exante API'))
        return strategy_result
        
    def update_tabel(self, strategy_result):
        'Шаг 3. Проверка и Запись в БД'
        connection = mysql.connector.connect(**CONFIG)
        cursor = connection.cursor()
        sql_2 = '''UPDATE tradeideas.unity_data SET 
            current_state = %s,
            current_price = %s,
            pl = %s,
            open_time = %s,
            close_time = %s,
            DURATION = %s,
            report = %s
        WHERE id = %s
        '''

        for row in strategy_result:
            if len(row) == 3:
                idx, value, report = row
                if isinstance(value,tuple):
                    value = list(value)
                    value.extend([report, idx])
                    cursor.execute(sql_2, tuple(value))
                
                else:
                    request = 'UPDATE tradeideas.unity_data SET current_state = %s, report = %s where id = %s'
                    cursor.execute(request, (value, report, idx))
                    
            else:
                idx, report = row
                request = 'UPDATE tradeideas.unity_data SET report = %s where id = %s'
                cursor.execute(request, (report, idx))
                
                
        # Подтвердите изменения
        connection.commit()

        # Закройте курсор и соединение
        cursor.close()
        connection.close()

                
        


# Create an instance of the Tester_Bot class
tester_bot_instance = Tester_Bot()

# Call the main_process_data method on the instance
result_df = tester_bot_instance.main_process_data()


ValueError: Length mismatch: Expected axis has 17 elements, new values have 11 elements

In [1]:
pd.set_option('display.max_columns', None)

NameError: name 'pd' is not defined

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import mysql.connector
import numpy as np

CONFIG = {
    'user': 'tester',
    'password': 'Fx123456#',
    'host': '192.168.1.6',
    'port': '3306',
    'raise_on_warnings': True}
UNITY_KEY = 'c3f28d5b-9694-4307-b30f-6f800ee13506'



def Unity_api(link:str)->list:
    key = 'c3f28d5b-9694-4307-b30f-6f800ee13506'
    headers = {'auth-token': key}
    response = requests.get(link, headers=headers)
    return response.json()
link = 'https://rest.unity.finance/api/v1/tradeIdeas?limit=1000&statuses=ACTIVE'
# link = 'https://rest.unity.finance/api/v1/instrumentDetails?limit=10'
d = Unity_api(link)
df = pd.DataFrame(d['items'])
# df['expirationTime'] = (pd.to_datetime(df['expirationTime'], format="%Y-%m-%dT%H:%M:%SZ")).dt.strftime("%Y-%m-%d %H:%M:%S")

df['expirationTime'] = pd.to_datetime(df['expirationTime'], format='mixed')
df['expirationTime'] = df['expirationTime'].dt.strftime("%Y-%m-%d %H:%M:%S")

df['createTime'] = pd.to_datetime(df['createTime'], format='mixed')
df['createTime'] = df['createTime'].dt.strftime("%Y-%m-%d %H:%M:%S")



df['instrumentIds'] = df['instrumentIds'].apply(lambda x: x[0])
# #===================================
instrumentIds = df['instrumentIds'].tolist()
instrumentIds = ",".join(map(str, instrumentIds))

link = f'https://rest.unity.finance/api/v1/instrumentDetails?instrumentId={instrumentIds}'
data = Unity_api(link)
ticker_df = pd.DataFrame(data['items'])
ticker_df = ticker_df[['id','code']]
df = pd.merge(df, ticker_df, left_on='instrumentIds', right_on='id', how='left')
df["instrumentIds"] = df['code']
# ============================

df.head(4)


,id_x,brokerId,instrumentIds,openOrderType,side,price,stopLoss,takeProfit,confidence,expirationTime,...,createTime,updateTime,source,provider,state,subscribers,priority,rate,id_y,code
0,5782,1,UBER.NYSE,LIMIT,BUY,62.94,61.87,66.00,20.0,2024-07-04 20:00:00,...,2024-06-04 11:49:33,2024-06-04T11:55:28Z,CONNECTOR,SIGNAL_CENTRE,None,0,None,None,11994,UBER.NYSE
1,5851,1,GOOGL.NASDAQ,LIMIT,BUY,174.54,170.92,183.59,60.0,2024-07-06 20:00:00,...,2024-06-06 13:16:09,2024-06-06T13:23:19Z,CONNECTOR,SIGNAL_CENTRE,None,0,None,None,11135,GOOGL.NASDAQ


In [15]:
instrumentIds

'7887,8115,8115,5213,1291,5213,48360,48360,59513,59626,59627,48001,59372,1670,48353,54,46,51,74,12,89,10,57,64,20228,41,47,48381,30,76,35,37,20225,20225,48345,26,53,83,59348,52,36,48360'

In [22]:
key = 'c3f28d5b-9694-4307-b30f-6f800ee13506'
headers = {'auth-token': key}
response = requests.get(f'https://rest.unity.finance/api/v1/instrumentDetails?instrumentId={instrumentIds}', headers=headers)
response.json()

{'items': [{'id': 47,
   'ticker': 'EURUSD',
   'code': 'EURUSD',
   'name': 'Euro vs US Dollar',
   'category': 'FX',
   'description': 'EUR (Euro) is the single currency of the European Union, the official monetary unit of 18 countries and has national status in 9 countries. It was introduced into non-cash circulation from January 1, 1999, and into cash - from January 1, 2002 in the form of coins and banknotes. Currency control is carried out by the EUCB. \n\nUSD (US Dollar) is the official currency of the United States, and also leads the world ranking of reserve assets. The right of issue belongs to the Fed. In countries where their own monetary unit does not act as the only means of payment, it is the "American" that is involved in circulation as the second main currency. At the moment, this pair is traded by the largest number of traders around the world. An important indicator in this value is the fact that the US dollar is the world currency and has maximum liquidity. The same 

In [28]:
# df = df.drop(['brokerId','status', 'publishTime','updateTime','id_y','code'], axis=1)
# df.head(4)

,id_x,instrumentIds,openOrderType,side,price,stopLoss,takeProfit,confidence,expirationTime,localeDescriptions,createTime,source,provider,state,subscribers,priority,rate
0,5118,BABA.NYSE,LIMIT,BUY,79.05,78.08,81.81,60.0,2024-06-07 20:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-05-07 11:14:37,CONNECTOR,SIGNAL_CENTRE,None,0,None,None
1,5777,GOOGL.NASDAQ,LIMIT,BUY,172.45,170.82,176.53,60.0,2024-06-05 20:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-06-04 09:49:38,CONNECTOR,SIGNAL_CENTRE,None,0,None,None
2,5782,UBER.NYSE,LIMIT,BUY,62.94,61.87,66.00,20.0,2024-07-04 20:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-06-04 11:49:33,CONNECTOR,SIGNAL_CENTRE,None,0,None,None
3,5790,UKOil,LIMIT,SELL,81.08,81.98,78.00,20.0,2024-06-06 05:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-06-05 05:21:34,CONNECTOR,SIGNAL_CENTRE,None,0,None,None


In [31]:
# df = df.drop(['brokerId','status', 'publishTime','updateTime','id_y','code'], axis=1)
# df.columns = [['unity_id','instrument','open_order_type','side', 'open_order_price','stop_loss', 'take_profit','confidence','expiration_time','description_text','create_time', 'source', 'provider', 'state', 'subscribers', 'priority','rate']]
#============================
df.head(4)

,unity_id,instrument,open_order_type,side,open_order_price,stop_loss,take_profit,confidence,expiration_time,description_text,create_time,source,provider,state,subscribers,priority,rate
0,5118,BABA.NYSE,LIMIT,BUY,79.05,78.08,81.81,60.0,2024-06-07 20:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-05-07 11:14:37,CONNECTOR,SIGNAL_CENTRE,None,0,None,None
1,5777,GOOGL.NASDAQ,LIMIT,BUY,172.45,170.82,176.53,60.0,2024-06-05 20:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-06-04 09:49:38,CONNECTOR,SIGNAL_CENTRE,None,0,None,None
2,5782,UBER.NYSE,LIMIT,BUY,62.94,61.87,66.00,20.0,2024-07-04 20:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-06-04 11:49:33,CONNECTOR,SIGNAL_CENTRE,None,0,None,None
3,5790,UKOil,LIMIT,SELL,81.08,81.98,78.00,20.0,2024-06-06 05:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-06-05 05:21:34,CONNECTOR,SIGNAL_CENTRE,None,0,None,None


In [26]:
table_name = 'tradeideas.unity_data'
connection = mysql.connector.connect(**CONFIG)
cursor = connection.cursor()

cursor.execute('SELECT unity_id FROM tradeideas.unity_data')
id_unity_spisok = np.array(cursor.fetchall()).flatten()
id_unity_spisok

array([2874, 2886, 2887, ..., 5821, 5822, 5823])

In [38]:
(((df['instrument'].iloc[25]).tolist()))

['Standard & Poor’s 500 Cash Index']

In [25]:


table_name = 'tradeideas.unity_data'
connection = mysql.connector.connect(**CONFIG)
cursor = connection.cursor()

# cursor.execute('SELECT unity_id FROM tradeideas.unity_data')
#
# id_unity_spisok = np.array(cursor.fetchall()).flatten()
columns_name = list(map(lambda x: x[0],df.columns.tolist()))
for index, row in df.iterrows():
    if len(((df['instrument'].iloc[index]).tolist())[0]) <= 35:

        values = [str(value) for value in row.tolist()]

        sql_query = f"INSERT INTO {table_name} ({', '.join(columns_name)}) VALUES ({', '.join(['%s']*len(columns_name))})"
        
        # Исполнение SQL-запроса
        cursor.execute(sql_query, tuple(values))

connection.commit()
cursor.close()
connection.close()

In [24]:
sql_query

'INSERT INTO tradeideas.unity_data (unity_id, instrument, open_order_type, side, open_order_price, stop_loss, take_profit, confidence, expiration_time, description_text, create_time, source, provider, state, subscribers, priority, rate) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [25]:
values

['5812',
 'Standard & Poor’s 500 Cash Index',
 'STOP',
 'SELL',
 '5295.0',
 '5315.0',
 '5250.0',
 '20.0',
 '2024-06-06 11:00:00',
 "[{'locale': 'RU', 'target': 'Мы собираемся продавать на пробое 5295', 'background': 'Восходящее движение является смешанным и волатильным – это обычное явление для корректирующих последовательностей\\nЦеновое действие сформировало медвежью фигуру «сужающийся клин»\\nФормация имеет измеренную цель движения 5250\\nОбласть слияния Фибоначчи находится в точке 5244\\nПробой 5295 необходим для подтверждения негативного импульса, продолжающегося со предыдущего дня'}, {'locale': 'EN', 'target': 'We look to Sell a break of 5295', 'background': 'The move higher is mixed and volatile, common in corrective sequences\\nPrice action has formed a bearish ending wedge formation\\nThe formation has a measured move target of 5250\\nA Fibonacci confluence area is located at 5244\\nA break of 5295 is needed to confirm follow through negative momentum'}]",
 '2024-06-05 09:38:3

In [26]:

columns_name

['unity_id',
 'instrument',
 'open_order_type',
 'side',
 'open_order_price',
 'stop_loss',
 'take_profit',
 'confidence',
 'expiration_time',
 'description_text',
 'create_time',
 'source',
 'provider',
 'state',
 'subscribers',
 'priority',
 'rate']

In [ ]:
'provider',
 'state',
 'subscribers',
 'priority',
 'rate'

In [319]:
df = df.iloc[9:]
df.head()

,unity_id,instrument,open_order_type,side,open_order_price,stop_loss,take_profit,confidence,expiration_time,description_text,create_time
9,3193,USDTRY,LIMIT,BUY,30.700,30.650,30.84,80.0,2024-02-16 21:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-02-16 06:11:16
10,3194,UKOil,LIMIT,SELL,83.270,84.270,80.44,60.0,2024-02-16 21:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-02-16 06:12:04
11,3195,EURSEK,LIMIT,BUY,11.260,11.230,11.34,40.0,2024-02-16 21:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-02-16 06:13:38
12,3196,USDJPY,LIMIT,BUY,149.040,148.540,150.32,80.0,2024-02-16 21:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-02-16 06:14:14
13,3197,USDCAD,LIMIT,BUY,1.345,1.343,1.35,80.0,2024-02-16 21:00:00,"[{'locale': 'RU', 'target': 'Мы намереваемся п...",2024-02-16 06:16:01


In [321]:
table_name = 'tradeideas.unity_data'
connection = mysql.connector.connect(**CONFIG)
cursor = connection.cursor()

# cursor.execute('SELECT unity_id FROM tradeideas.unity_data')
#
# id_unity_spisok = np.array(cursor.fetchall()).flatten()
columns_name = list(map(lambda x: x[0],df.columns.tolist()))
for index, row in df.iterrows():
    # unity_id = row['unity_id']
    # if unity_id not in id_unity_spisok:
    values = [str(value) for value in row.tolist()]

    sql_query = f"INSERT INTO {table_name} ({', '.join(columns_name)}) VALUES ({', '.join(['%s']*len(columns_name))})"
    
    # Исполнение SQL-запроса
    cursor.execute(sql_query, tuple(values))

connection.commit()
cursor.close()
connection.close()


In [307]:
sql_query

'INSERT INTO tradeideas.unity_data (unity_id, instrument, open_order_type, side, open_order_price, stop_loss, take_profit, confidence, expiration_time, description_text, create_time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [322]:
values

['3220',
 'XRPUSD',
 'LIMIT',
 'BUY',
 '0.546',
 '0.534',
 '0.576',
 '60.0',
 '2024-02-16 21:00:00',
 '[{\'locale\': \'RU\', \'target\': \'Мы намереваемся покупать на 0.5460\', \'background\': \'Наше краткосрочное смещение остается позитивным\\nВчерашний Марабузо находится на 0.5460\\nОжидается нижняя коррекция\\nМы хотим купить откаты\\nОбъем продаж увеличивается\\n20 4часовая экспоненциальная скользящая средняя находится на 0.5426\'}, {\'locale\': \'EN\', \'target\': \'We look to Buy at 0.5460\', \'background\': "Our short term bias remains positive\\nYesterday\'s Marabuzo is located at 0.5460\\nA lower correction is expected\\nWe look to buy dips\\nTrading volume is increasing\\n20 4hour EMA is at 0.5426"}]',
 '2024-02-16 11:17:30']

In [156]:
table_name = 'tradeideas.unity_data'
connection = mysql.connector.connect(**CONFIG)
cursor = connection.cursor()

cursor.execute('SELECT unity_id FROM tradeideas.unity_data')
id_unity_spisok = np.array(cursor.fetchall()).flatten()

columns_name = list(map(lambda x: x[0],df.columns.tolist()))
# columns_name = df2.columns.tolist()
for index, row in df.iterrows():
    unity_id = row['unity_id']
    if unity_id not in id_unity_spisok:
        values = [str(value) for value in row.tolist()]

        sql_query = f"INSERT INTO {table_name} ({', '.join(columns_name)}) VALUES ({', '.join(['%s']*len(columns_name))})"
        
        # Исполнение SQL-запроса
        cursor.execute(sql_query, tuple(values))
    
    
# Подтверждение изменений в базе данных
connection.commit()

# Закрытие курсора и соединения
cursor.close()
connection.close()

# Шаг 2. Забираем нужные нам данные из БД

In [67]:
connection = mysql.connector.connect(**CONFIG)
sql_query = '''
SELECT id, instrument, side, open_order_type, open_order_price, stop_loss, take_profit, create_time, expiration_time
FROM tradeideas.unity_data
WHERE 
expiration_time <= NOW() and 
report is NULL;'''
check_df = pd.read_sql(sql_query, connection)
connection.close()
check_df.head()

C:\Temp\ipykernel_14360\2243296513.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  check_df = pd.read_sql(sql_query, connection)


,id,instrument,side,open_order_type,open_order_price,stop_loss,take_profit,create_time,expiration_time
0,7,AUDJPY,BUY,LIMIT,96.000,95.500,97.2500,2024-02-05 05:41:26,2024-02-06 06:00:00
1,8,EURSEK,BUY,LIMIT,11.300,11.270,11.3800,2024-02-05 05:46:40,2024-02-06 06:00:00
2,9,USDCAD,SELL,STOP,1.346,1.349,1.3395,2024-02-05 05:48:29,2024-02-06 06:00:00
3,10,USDTRY,BUY,LIMIT,30.400,30.350,30.5400,2024-02-05 06:03:31,2024-02-06 06:00:00
4,11,UKOil,BUY,LIMIT,75.300,74.100,80.0000,2024-02-05 06:06:00,2024-02-06 06:00:00


In [68]:
# ИЗМЕНЯЕМ ВРЕМЯ НА НОРМАЛЬНОЕ 
check_df['expiration_time'] = (pd.to_datetime(check_df['expiration_time'], format="%Y-%m-%d %H:%M:%S")).dt.strftime("%d.%m.%Y %H:%M:%S")
check_df['create_time'] = (pd.to_datetime(check_df['create_time'], format="%Y-%m-%d %H:%M:%S")).dt.strftime("%d.%m.%Y %H:%M:%S")
check_df.set_index('id', inplace=True)
check_df

,instrument,side,open_order_type,open_order_price,stop_loss,take_profit,create_time,expiration_time
id,,,,,,,,
7,AUDJPY,BUY,LIMIT,96.0000,95.5000,97.2500,05.02.2024 05:41:26,06.02.2024 06:00:00
8,EURSEK,BUY,LIMIT,11.3000,11.2700,11.3800,05.02.2024 05:46:40,06.02.2024 06:00:00
9,USDCAD,SELL,STOP,1.3460,1.3490,1.3395,05.02.2024 05:48:29,06.02.2024 06:00:00
10,USDTRY,BUY,LIMIT,30.4000,30.3500,30.5400,05.02.2024 06:03:31,06.02.2024 06:00:00
11,UKOil,BUY,LIMIT,75.3000,74.1000,80.0000,05.02.2024 06:06:00,06.02.2024 06:00:00
12,USDJPY,BUY,LIMIT,147.2800,146.7800,148.5000,05.02.2024 06:09:34,06.02.2024 06:00:00
13,EURUSD,BUY,LIMIT,1.0763,1.0733,1.0855,05.02.2024 06:13:49,06.02.2024 06:00:00
14,XPTUSD,SELL,LIMIT,910.0000,920.0000,886.0000,05.02.2024 06:22:28,06.02.2024 06:00:00
15,XAUUSD,BUY,LIMIT,2016.5000,2004.2500,2051.5000,05.02.2024 06:18:19,06.02.2024 06:00:00


In [51]:
# check_df = pd.DataFrame(["EURSEK",	"Buy",	"Limit", 11.338,	11.303,	11.408,	"25.01.2024 06:48:48",	"26.01.2024 09:00:00"])

# check_df = check_df.T
# check_df.columns = ["instrument",	"side",	"open_order_type",	"open_order_price",	"stop_loss",	"take_profit",	"create_time",	"expiration_time"]
# check_df

,instrument,side,open_order_type,open_order_price,stop_loss,take_profit,create_time,expiration_time
0,EURSEK,Buy,Limit,11.338,11.303,11.408,25.01.2024 06:48:48,26.01.2024 09:00:00


## Сама проверка

In [157]:
from process_data import load_data
import time
import json
with open('right_instumets.json', 'r', encoding='utf-8') as f:
    RIGHT_INSTRUMENTS = json.load(f)

In [158]:
data = load_data('NVDA.NASDAQ','09.02.2024 13:43:31', '09.02.2024 21:00:00')



In [160]:
df = pd.DataFrame(data)
df['timestamp2'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_localize('UTC')
# Преобразуем дату в формат "d-m-Y H:M:S" и создадим новый столбец 'formatted_date'
df['formatted_date'] = df['timestamp2'].dt.strftime('%Y-%m-%d %H:%M:%S')
df = df.drop(columns=["timestamp2"])
df[["close", "high", "low", "open"]] = df[["close", "high", "low", "open"]].apply(pd.to_numeric, errors='coerce')
df = df[::-1]
df = df.reset_index(drop=True)
df.head()

,close,high,low,open,timestamp,formatted_date
0,704.105,704.165,703.780,703.785,1707486180000,2024-02-09 13:43:00
1,704.150,704.305,704.005,704.100,1707486240000,2024-02-09 13:44:00
2,703.890,704.215,703.890,704.140,1707486300000,2024-02-09 13:45:00
3,704.015,704.015,703.685,703.880,1707486360000,2024-02-09 13:46:00
4,704.060,704.060,703.880,704.010,1707486420000,2024-02-09 13:47:00


In [161]:
df.tail()

,close,high,low,open,timestamp,formatted_date
431,720.740,720.900,720.630,720.815,1707512040000,2024-02-09 20:54:00
432,721.830,721.830,720.705,720.705,1707512100000,2024-02-09 20:55:00
433,721.665,721.825,721.190,721.825,1707512160000,2024-02-09 20:56:00
434,721.075,721.770,721.075,721.690,1707512220000,2024-02-09 20:57:00
435,721.405,721.640,721.070,721.070,1707512280000,2024-02-09 20:58:00


In [60]:
def Strategy_Tester(data:list, side:str, open_order_type:str, stop_loss:float, price:float, take_profit:float)->tuple:
    df = pd.DataFrame(data)
    # Преобразование timestamp в формат datetime с учетом часового пояса UTC
    df['timestamp2'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_localize('UTC')
    # Преобразуем дату в формат "d-m-Y H:M:S" и создадим новый столбец 'formatted_date'
    df['formatted_date'] = df['timestamp2'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df = df.drop(columns=["timestamp2"])
    df[["close", "high", "low", "open"]] = df[["close", "high", "low", "open"]].apply(pd.to_numeric, errors='coerce')
    df = df[::-1]
    df = df.reset_index(drop=True)
    '''Алгоритм проверки'''
    new_df = df.copy()
    PRICE_INDICATOR = False
    STOP_LOSS_INDICATOR = False
    TAKE_PROFIT_INDICATOR = False
    # 1. Проверяем вход по цене
    if side == 'Buy':
        Flag=True
        if open_order_type == 'Limit':
            price_open = new_df[new_df['low'] <= price]
        else:
            price_open = new_df[new_df['high'] >= price]
    else:
        Flag = False
        if open_order_type == 'Limit':
            price_open = new_df[new_df['high'] >= price]
        else:
            price_open = new_df[new_df['low'] <= price]

    if not price_open.empty:
        index_price = price_open.index[0]
        new_df = new_df.iloc[index_price:].reset_index(drop=True)
        PRICE_INDICATOR = True
        approximate_time_enter, accurate_time_enter = price_open['formatted_date'].iloc[0], price_open['timestamp'].iloc[0]
    # 2. Проверяем стоп-лосс
    if Flag:
        stop_loss_res = new_df[new_df['low'] <= stop_loss]
    else:
        stop_loss_res = new_df[new_df['high'] >= stop_loss]

    if not stop_loss_res.empty:
        index_stop_loss = stop_loss_res.index[0]
        new_df = new_df.iloc[:index_stop_loss + 1].reset_index(drop=True)
        STOP_LOSS_INDICATOR = True
        approximate_time_ST, accurate_time_ST = stop_loss_res['formatted_date'].iloc[0], stop_loss_res['timestamp'].iloc[0]

    # 3. Проверяем на take profit
    if Flag:
        take_profit_res = new_df[new_df['high'] >= take_profit]
    else:
        take_profit_res = new_df[new_df['low'] <= take_profit]
        
    if not take_profit_res.empty:
        take_profit_index = take_profit_res.index[0]
        TAKE_PROFIT_INDICATOR = True
        approximate_time_TP, accurate_time_TP = take_profit_res['formatted_date'].iloc[0], take_profit_res['timestamp'].iloc[0]

    def timedelta(str1, str2):
        date_format = '%Y-%m-%d %H:%M:%S'
        date1 = datetime.strptime(str1, date_format)
        date2 = datetime.strptime(str2, date_format)
        # Вычисление разницы между датами
        time_difference = date2 - date1 
        # Извлечение часов, минут и секунд из разницы
        hours, remainder = divmod(time_difference.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        # Форматирование результатов
        result_string = '{:02}:{:02}:{:02}'.format(hours, minutes, seconds)
        return result_string
        
    # 4. Вывод
    if PRICE_INDICATOR:
        if TAKE_PROFIT_INDICATOR:
            return("TP",take_profit, abs(take_profit-price), approximate_time_enter, approximate_time_TP, timedelta(approximate_time_enter,approximate_time_TP))
        elif STOP_LOSS_INDICATOR and not TAKE_PROFIT_INDICATOR:
            return ("SL",stop_loss, -1*abs(stop_loss-price), approximate_time_enter, approximate_time_ST, timedelta(approximate_time_enter,approximate_time_ST))
        else:
            last_price = new_df['close'].iloc[-1]
            approximate_time_EXP = new_df['formatted_date'].iloc[-1]
            return ("EXP", 
                    last_price, 
                    last_price - price if side == 'Buy'else price - last_price, 
                    approximate_time_enter, 
                    approximate_time_EXP, 
                    timedelta(approximate_time_enter,approximate_time_EXP))
    else:
        return('NOT_ENTER')

In [69]:
check_df = check_df.iloc[:5]
check_df

,instrument,side,open_order_type,open_order_price,stop_loss,take_profit,create_time,expiration_time
id,,,,,,,,
7,AUDJPY,BUY,LIMIT,96.000,95.500,97.2500,05.02.2024 05:41:26,06.02.2024 06:00:00
8,EURSEK,BUY,LIMIT,11.300,11.270,11.3800,05.02.2024 05:46:40,06.02.2024 06:00:00
9,USDCAD,SELL,STOP,1.346,1.349,1.3395,05.02.2024 05:48:29,06.02.2024 06:00:00
10,USDTRY,BUY,LIMIT,30.400,30.350,30.5400,05.02.2024 06:03:31,06.02.2024 06:00:00
11,UKOil,BUY,LIMIT,75.300,74.100,80.0000,05.02.2024 06:06:00,06.02.2024 06:00:00


In [70]:
strategy_result = []
for idx, row in check_df.iterrows():
    instrument, side, open_order_type, open_order_price, stop_loss, take_profit, create_time, expiration_time  = row
    if instrument in RIGHT_INSTRUMENTS:
        instrument = RIGHT_INSTRUMENTS[instrument]
    try:
        data = load_data(instrument, create_time, expiration_time)
        result = Strategy_Tester(data, side, open_order_type, stop_loss, open_order_price, take_profit)
        strategy_result.append((idx, result, 'OK'))
        time.sleep(60)   
    except:
        time.sleep(60)
        strategy_result.append((idx, 'Неизвестный запрос для Exante API'))
strategy_result

[(7, 'NOT_ENTER', 'OK'),
 (8, 'NOT_ENTER', 'OK'),
 (9, 'NOT_ENTER', 'OK'),
 (10, 'NOT_ENTER', 'OK'),
 (11, 'Неизвестный запрос для Exante API')]

In [139]:
data = load_data('AMZN.NASDAQ', '07.02.2024 14:17:36', '09.02.2024 21:00:00')

In [140]:
df = pd.DataFrame(data)
# Преобразование timestamp в формат datetime с учетом часового пояса UTC
df['timestamp2'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_localize('UTC')
# Преобразуем дату в формат "d-m-Y H:M:S" и создадим новый столбец 'formatted_date'
df['formatted_date'] = df['timestamp2'].dt.strftime('%Y-%m-%d %H:%M:%S')
df = df.drop(columns=["timestamp2"])
df[["close", "high", "low", "open"]] = df[["close", "high", "low", "open"]].apply(pd.to_numeric, errors='coerce')
df = df[::-1]
df = df.reset_index(drop=True)
df

,close,high,low,open,timestamp,formatted_date
0,169.765,169.795,169.755,169.775,1707315420000,2024-02-07 14:17:00
1,169.780,169.780,169.755,169.755,1707315480000,2024-02-07 14:18:00
2,169.700,169.770,169.700,169.765,1707315540000,2024-02-07 14:19:00
3,169.715,169.715,169.665,169.695,1707315600000,2024-02-07 14:20:00
4,169.775,169.800,169.725,169.725,1707315660000,2024-02-07 14:21:00
...,...,...,...,...,...,...
2068,174.525,174.585,174.475,174.545,1707512100000,2024-02-09 20:55:00
2069,174.365,174.515,174.365,174.515,1707512160000,2024-02-09 20:56:00
2070,174.315,174.385,174.305,174.385,1707512220000,2024-02-09 20:57:00
2071,174.385,174.395,174.255,174.305,1707512280000,2024-02-09 20:58:00


In [32]:
# strategy_result = [(41, 'Неизвестный запрос для Exante API'),
#  (42, 'NOT', 'OK'),
#  (43, 'NOT', 'OK'),
#  (44,
#   ('SL',
#    11.303,
#    -0.034999999999998366,
#    '2024-01-25 12:34:00',
#    '2024-01-26 10:32:00',
#    '21:58:00'),
#   'OK')]
# strategy_result

[(41, 'Неизвестный запрос для Exante API'),
 (42, 'NOT', 'OK'),
 (43, 'NOT', 'OK'),
 (44,
  ('SL',
   11.303,
   -0.034999999999998366,
   '2024-01-25 12:34:00',
   '2024-01-26 10:32:00',
   '21:58:00'),
  'OK')]

# Шаг 3. Запись в БД

In [71]:
connection = mysql.connector.connect(**CONFIG)
cursor = connection.cursor()
sql_2 = '''UPDATE tradeideas.unity_data SET 
    current_state = %s,
    current_price = %s,
    pl = %s,
    open_time = %s,
    close_time = %s,
    DURATION = %s,
    report = %s
WHERE id = %s
'''

for row in strategy_result:
    if len(row) == 3:
        idx, value, report = row
        if isinstance(value,tuple):
            value = list(value)
            value.extend([report, idx])
            cursor.execute(sql_2, tuple(value))
            print (tuple(value))
        else:
            request = 'UPDATE tradeideas.unity_data SET current_state = %s, report = %s where id = %s'
            cursor.execute(request, (value, report, idx))
            print((value, report, idx))

    else:
        idx, report = row
        request = 'UPDATE tradeideas.unity_data SET report = %s where id = %s'
        cursor.execute(request, (report, idx))
        print((report, idx))
        
# Подтвердите изменения
connection.commit()

# Закройте курсор и соединение
cursor.close()
connection.close()

('NOT_ENTER', 'OK', 7)
('NOT_ENTER', 'OK', 8)
('NOT_ENTER', 'OK', 9)
('NOT_ENTER', 'OK', 10)
('Неизвестный запрос для Exante API', 11)


In [29]:
# Подтвердите изменения
connection.commit()

# Закройте курсор и соединение
cursor.close()
connection.close()

## Шаг 4. Отчет

In [162]:
CONFIG = {
    'user': 'tester',
    'password': 'Fx123456#',
    'host': '192.168.1.6',
    'port': '3306',
    'raise_on_warnings': True}

In [167]:
connection = mysql.connector.connect(**CONFIG)
sql = '''
SELECT expiration_time, current_state, report,pl
FROM tradeideas.unity_data
WHERE 
expiration_time BETWEEN DATE_SUB(CURDATE(), INTERVAL 1 DAY) AND CURDATE();'''
df = pd.read_sql(sql,connection)
connection.close()
df.head()

C:\Temp\ipykernel_14360\2141044112.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,connection)


,expiration_time,current_state,report,pl
0,2024-02-13 06:00:00,NOT_ENTER,OK,NaN
1,2024-02-13 06:00:00,NOT_ENTER,OK,NaN
2,2024-02-13 06:00:00,SL,OK,-0.03
3,2024-02-13 06:00:00,NOT_ENTER,OK,NaN
4,2024-02-13 06:00:00,NOT_ENTER,OK,NaN


In [166]:
total_records = len(df)
dct= df['current_state'].value_counts().to_dict()
unknow_methods = df['report'].value_counts()['Неизвестный запрос для Exante API']
EXP_positiv = len(df[(df['current_state'] == 'EXP') & (df['pl'] > 0)])
date = str(df['expiration_time'].iloc[0])[:10]

KeyError: 'Неизвестный запрос для Exante API'

In [132]:
from process_data import send_message

In [133]:
s = f'''Всего идей проверенно {total_records} на момент {date}\n
Статистика - {dct}\n
Неизвечтный метод API EXANTE - {unknow_methods}\n
Experation ideas positiv - {EXP_positiv}'''
print(s)

Всего идей проверенно 35 на момент 2024-02-08

Статистика - {'NOT_ENTER': 24, 'EXP': 4, 'SL': 1}

Неизвечтный метод API EXANTE - 6

Experation ideas positiv - 3


In [134]:
dct.get('TP',0)

0

In [135]:
dct

{'NOT_ENTER': 24, 'EXP': 4, 'SL': 1}

In [136]:
send_message(s)

In [117]:
def report_every_day():
    connection = mysql.connector.connect(**CONFIG)
    sql = '''
    SELECT expiration_time, current_state, report,pl
    FROM tradeideas.unity_data
    WHERE 
    expiration_time BETWEEN DATE_SUB(CURDATE(), INTERVAL 1 DAY) AND CURDATE();'''
    df = pd.read_sql(sql,connection)
    connection.close()
    #==================
    total_records = len(df)
    dct= df['current_state'].value_counts().to_dict()
    unknow_methods = df['report'].value_counts()['Неизвестный запрос для Exante API']
    EXP_positiv = len(df[(df['current_state'] == 'EXP') & (df['pl'] > 0)])
    date = str(df['expiration_time'].iloc[0])[:10]
    #==================
    s = f'''Всего идей проверенно {total_records} на момент {date}\n
Статистика - {dct}\n
Неизвечтный метод API EXANTE - {unknow_methods}\n
Experation ideas positiv - {EXP_positiv}'''
    send_message(s)

report_every_day()

C:\Temp\ipykernel_14360\239794753.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,connection)


In [137]:
a = f'''Всего получено сигналов: {total_records} за {date}
Из них:
- Успешно прошли проверку: {total_records-unknow_methods}
- Нет инструмента на Exante: {unknow_methods}

По итогу проверки из {total_records-unknow_methods} прошедших проверку:
- Take profit = {dct.get('TP',0)}
- Stop Loss = {dct.get('SL',0)}
- Не сработали вовсе = {dct.get('NOT_ENTER',0)}
- На момент экспирации была открытая позиция = {dct.get('EXP',0)}
- Из них {EXP_positiv} закрыты в Profit и {dct.get('EXP',0) - EXP_positiv} в Loss'''
print(a)

Всего получено сигналов: 35 за 2024-02-08
Из них:
- Успешно прошли проверку: 29
- Нет инструмента на Exante: 6

По итогу проверки из 29 прошедших проверку:
- Take profit = 0
- Stop Loss = 1
- Не сработали вовсе = 24
- На момент экспирации была открытая позиция = 4
- Из них 3 закрыты в Profit и 1 в Loss


In [138]:
send_message(a)

## Повторная проверка EXP сделкок

In [183]:
connection = mysql.connector.connect(**CONFIG)
# id, instrument, side, open_order_type, open_order_price, stop_loss, take_profit, create_time, expiration_time
sql = '''
SELECT id, unity_id,  instrument, side, open_order_type, open_order_price, stop_loss, take_profit, current_state, pl, create_time, expiration_time, close_time
FROM tradeideas.unity_data
WHERE current_state = 'EXP'
AND TIMESTAMPDIFF(MINUTE, close_time, expiration_time) > 1'''
df = pd.read_sql(sql,connection)
connection.close()


C:\Temp\ipykernel_14360\110394177.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,connection)


In [184]:
df['Delta_time'] = df["expiration_time"] - df["close_time"]
df.head()

,id,unity_id,instrument,side,open_order_type,open_order_price,stop_loss,take_profit,current_state,pl,create_time,expiration_time,close_time,Delta_time
0,61,2936,UK100,SELL,LIMIT,7690.000,7727.000,7585.000,EXP,-7.00000,2024-02-06 07:14:15,2024-02-07 06:00:00,2024-02-06 16:59:00,0 days 13:01:00
1,72,2947,ETHUSD,SELL,LIMIT,2375.000,2435.000,2225.000,EXP,8.00050,2024-02-06 12:22:27,2024-02-07 12:00:00,2024-02-07 10:46:00,0 days 01:14:00
2,78,2953,USDMXN,BUY,LIMIT,17.025,16.975,17.145,EXP,0.00947,2024-02-06 17:02:11,2024-02-07 06:00:00,2024-02-07 05:55:00,0 days 00:05:00
3,82,2957,EURSEK,BUY,LIMIT,11.260,11.230,11.340,EXP,0.02536,2024-02-07 05:26:06,2024-02-08 06:00:00,2024-02-08 04:02:00,0 days 01:58:00
4,89,2964,XAGUSD,BUY,LIMIT,22.250,22.090,22.705,EXP,-0.01450,2024-02-07 06:46:33,2024-02-08 06:00:00,2024-02-08 04:09:00,0 days 01:51:00


In [179]:
df.to_excel('EXP сделки до доп проверки.xlsx')

In [187]:
from settings import PARAMETERS


def load_data_2(symbol_id:str, CREATE_TIME:str, EXPIRATION_TIME:str)->list:
    def transfrm_to_TIMESTAMP(date_string:str)->str:
        date = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S')
        return int(date.timestamp()) * 1000 
    start_time =  transfrm_to_TIMESTAMP(CREATE_TIME)
    end_time = transfrm_to_TIMESTAMP(EXPIRATION_TIME)
    duration = 60 
    size = int(round(((end_time - start_time)/1000/duration), 1))
    url = f'https://api-live.exante.eu/md/2.0/ohlc/{symbol_id}/{duration}'
    params = {
    'from': start_time, 
    'to': end_time, 
    'size': size,
    'type': 'quotes'}
    response = requests.get(url, auth=(PARAMETERS["API Key"], PARAMETERS["Secret Key"]), params=params)
    data = json.loads(response.text)
    return data

In [192]:
load_data_2('USD%2FCHF.E.FX','2024-02-07 06:46:33','2024-02-08 06:00:00')

[{'close': '0.87313',
  'high': '0.87322',
  'low': '0.87312',
  'open': '0.873215',
  'timestamp': 1707371940000},
 {'close': '0.87321',
  'high': '0.87325',
  'low': '0.873205',
  'open': '0.87325',
  'timestamp': 1707371880000},
 {'close': '0.87324',
  'high': '0.87327',
  'low': '0.87321',
  'open': '0.873225',
  'timestamp': 1707371820000},
 {'close': '0.87323',
  'high': '0.873255',
  'low': '0.8732',
  'open': '0.873245',
  'timestamp': 1707371760000},
 {'close': '0.87324',
  'high': '0.87342',
  'low': '0.87324',
  'open': '0.873405',
  'timestamp': 1707371700000},
 {'close': '0.8734',
  'high': '0.873465',
  'low': '0.8734',
  'open': '0.873455',
  'timestamp': 1707371640000},
 {'close': '0.87345',
  'high': '0.87346',
  'low': '0.873355',
  'open': '0.873415',
  'timestamp': 1707371580000},
 {'close': '0.87341',
  'high': '0.873465',
  'low': '0.87336',
  'open': '0.873365',
  'timestamp': 1707371520000},
 {'close': '0.87337',
  'high': '0.87339',
  'low': '0.87329',
  'open'

In [194]:
row[['instrument', "side", "open_order_type", "open_order_price", "stop_loss", "take_profit", "create_time", "expiration_time"]]

instrument                        UK100
side                               SELL
open_order_type                   LIMIT
open_order_price                 7690.0
stop_loss                        7727.0
take_profit                      7585.0
create_time         2024-02-06 07:14:15
expiration_time     2024-02-07 06:00:00
Name: 0, dtype: object

In [212]:
'AJKJA'.capitalize()

'Ajkja'

In [213]:
# Проверка

strategy_result = []
for idx, row in df.iterrows():

    instrument, side, open_order_type, open_order_price, stop_loss, take_profit, create_time, expiration_time  = row[['instrument', "side", "open_order_type", "open_order_price", "stop_loss", "take_profit", "create_time", "expiration_time"]]
    create_time, expiration_time = str(create_time), str(expiration_time)
    if instrument in RIGHT_INSTRUMENTS:
        instrument = RIGHT_INSTRUMENTS[instrument]
    side, open_order_type = side.capitalize(), open_order_type.capitalize()
    try:
        data = load_data_2(instrument, create_time, expiration_time)
        result = Strategy_Tester(data, side, open_order_type, stop_loss, open_order_price, take_profit)
        strategy_result.append((result, 'OK'))
        time.sleep(60)   
    except:
        time.sleep(60)
        strategy_result.append(('Неизвестный запрос для Exante API'))
    # print(idx, strategy_result)
strategy_result

[(('EXP',
   7697.0,
   -7.0,
   '2024-02-06 08:00:00',
   '2024-02-06 16:59:00',
   '08:59:00'),
  'OK'),
 (('EXP',
   2361.4515,
   13.548499999999876,
   '2024-02-06 19:33:00',
   '2024-02-07 11:58:00',
   '16:25:00'),
  'OK'),
 (('EXP',
   17.03447,
   0.009470000000000312,
   '2024-02-06 17:18:00',
   '2024-02-07 05:55:00',
   '12:37:00'),
  'OK'),
 (('EXP',
   11.2864,
   0.026400000000000645,
   '2024-02-07 13:24:00',
   '2024-02-08 05:58:00',
   '16:34:00'),
  'OK'),
 (('EXP',
   22.329,
   0.07900000000000063,
   '2024-02-07 08:28:00',
   '2024-02-08 05:59:00',
   '21:31:00'),
  'OK'),
 (('EXP',
   44698.975,
   890.9749999999985,
   '2024-02-07 20:05:00',
   '2024-02-08 11:59:00',
   '15:54:00'),
  'OK'),
 (('EXP',
   174.445,
   1.7949999999999875,
   '2024-02-09 15:37:00',
   '2024-02-09 20:59:00',
   '05:22:00'),
  'OK'),
 (('EXP',
   0.608825,
   0.003175000000000039,
   '2024-02-08 02:06:00',
   '2024-02-08 14:59:00',
   '12:53:00'),
  'OK'),
 (('EXP',
   1.346485,
   0.

In [222]:
name = 'current_state, current_price, pl, open_time, close_time, DURATION'
name = [i+'_after_test' for i in name.split(',')]
name

['current_state_after_test',
 ' current_price_after_test',
 ' pl_after_test',
 ' open_time_after_test',
 ' close_time_after_test',
 ' DURATION_after_test']

In [223]:
new_r = list(map(lambda x: x[0],strategy_result))
new_r
df3 = pd.DataFrame(new_r)
df3.columns = name
df3

,current_state_after_test,current_price_after_test,pl_after_test,open_time_after_test,close_time_after_test,DURATION_after_test
0,EXP,7697.000000,-7.000000,2024-02-06 08:00:00,2024-02-06 16:59:00,08:59:00
1,EXP,2361.451500,13.548500,2024-02-06 19:33:00,2024-02-07 11:58:00,16:25:00
2,EXP,17.034470,0.009470,2024-02-06 17:18:00,2024-02-07 05:55:00,12:37:00
3,EXP,11.286400,0.026400,2024-02-07 13:24:00,2024-02-08 05:58:00,16:34:00
4,EXP,22.329000,0.079000,2024-02-07 08:28:00,2024-02-08 05:59:00,21:31:00
5,EXP,44698.975000,890.975000,2024-02-07 20:05:00,2024-02-08 11:59:00,15:54:00
6,EXP,174.445000,1.795000,2024-02-09 15:37:00,2024-02-09 20:59:00,05:22:00
7,EXP,0.608825,0.003175,2024-02-08 02:06:00,2024-02-08 14:59:00,12:53:00
8,EXP,1.346485,0.001515,2024-02-08 11:25:00,2024-02-09 05:59:00,18:34:00
9,EXP,22.603000,0.273000,2024-02-08 07:47:00,2024-02-09 05:59:00,22:12:00


In [227]:
# df = df.drop(['expiration_time_str','create_time_str'], axis=1)
# df

In [235]:
# Выполняем левое соединение
# Используем concat для объединения по столбцам
result_df = pd.concat([df, df3], axis=1)
result_df.head()

,id,unity_id,instrument,side,open_order_type,open_order_price,stop_loss,take_profit,current_state,pl,create_time,expiration_time,close_time,Delta_time,current_state_after_test,current_price_after_test,pl_after_test,open_time_after_test,close_time_after_test,DURATION_after_test
0,61,2936,UK100,SELL,LIMIT,7690.000,7727.000,7585.000,EXP,-7.00000,2024-02-06 07:14:15,2024-02-07 06:00:00,2024-02-06 16:59:00,0 days 13:01:00,EXP,7697.00000,-7.00000,2024-02-06 08:00:00,2024-02-06 16:59:00,08:59:00
1,72,2947,ETHUSD,SELL,LIMIT,2375.000,2435.000,2225.000,EXP,8.00050,2024-02-06 12:22:27,2024-02-07 12:00:00,2024-02-07 10:46:00,0 days 01:14:00,EXP,2361.45150,13.54850,2024-02-06 19:33:00,2024-02-07 11:58:00,16:25:00
2,78,2953,USDMXN,BUY,LIMIT,17.025,16.975,17.145,EXP,0.00947,2024-02-06 17:02:11,2024-02-07 06:00:00,2024-02-07 05:55:00,0 days 00:05:00,EXP,17.03447,0.00947,2024-02-06 17:18:00,2024-02-07 05:55:00,12:37:00
3,82,2957,EURSEK,BUY,LIMIT,11.260,11.230,11.340,EXP,0.02536,2024-02-07 05:26:06,2024-02-08 06:00:00,2024-02-08 04:02:00,0 days 01:58:00,EXP,11.28640,0.02640,2024-02-07 13:24:00,2024-02-08 05:58:00,16:34:00
4,89,2964,XAGUSD,BUY,LIMIT,22.250,22.090,22.705,EXP,-0.01450,2024-02-07 06:46:33,2024-02-08 06:00:00,2024-02-08 04:09:00,0 days 01:51:00,EXP,22.32900,0.07900,2024-02-07 08:28:00,2024-02-08 05:59:00,21:31:00


In [269]:
# # Перед удалением столбцов, проверим, существуют ли они в DataFrame
# print(result_df.columns)

# # Использование errors='ignore' для предотвращения KeyError, если столбцы отсутствуют
# result_df = result_df.drop([' current_price_after_test', ' DURATION_after_test'], axis=1, errors='ignore')

# result_df['close_time_after_test'] = pd.to_datetime(result_df[' close_time_after_test'])

# result_df['Delta_time_after_test'] = result_df['expiration_time'] - result_df['close_time_after_test']
# result_df['Delta_time'] = result_df['expiration_time'] - result_df['close_time']
# # Просмотрим первые две строки, чтобы убедиться, что операция прошла успешно
# result_df


In [271]:
# Перед удалением столбцов, проверим, существуют ли они в DataFrame
print(result_df.columns)

# Использование errors='ignore' для предотвращения KeyError, если столбцы отсутствуют
result_df = result_df.drop([' close_time_after_test'], axis=1, errors='ignore')


Index(['id', 'unity_id', 'instrument', 'current_state', 'pl', 'create_time',
       'expiration_time', 'close_time', 'Delta_time',
       'current_state_after_test', ' pl_after_test', ' open_time_after_test',
       ' close_time_after_test', 'close_time_after_test',
       'Delta_time_after_test'],
      dtype='object')


In [274]:
result_df.to_excel('Отчет по EXP сделкам.xlsx')

In [275]:
result_df.head(2)

,id,unity_id,instrument,current_state,pl,create_time,expiration_time,close_time,Delta_time,current_state_after_test,pl_after_test,open_time_after_test,close_time_after_test,Delta_time_after_test
0,61,2936,UK100,EXP,-7.0000,2024-02-06 07:14:15,2024-02-07 06:00:00,2024-02-06 16:59:00,0 days 13:01:00,EXP,-7.0000,2024-02-06 08:00:00,2024-02-06 16:59:00,0 days 13:01:00
1,72,2947,ETHUSD,EXP,8.0005,2024-02-06 12:22:27,2024-02-07 12:00:00,2024-02-07 10:46:00,0 days 01:14:00,EXP,13.5485,2024-02-06 19:33:00,2024-02-07 11:58:00,0 days 00:02:00
